In [1]:
!nvidia-smi

Mon Jan 11 03:28:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip3 install nltk
import nltk
nltk.download('punkt')

!pip3 install toposort

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#import os
#import urllib.request
#urllib.request.urlretrieve('https://storage.googleapis.com/albert_models/albert_base_zh.tar.gz','albert_base_zh.tar.gz')

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/colab_data/packages/albert_QA_colab')
#sys.path=['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']
print(sys.path)

['/tensorflow-1.15.2/python3.6', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/colab_data/packages/albert_QA_colab']


In [5]:
import os

# AMP要使用 tf.keras 
os.environ["TF_KERAS"] = "1"
# 重计算设置
import memory_saving_gradients as gc
from tensorflow.python.ops import gradients as tf_gradients
tf_gradients.gradients = gc.gradients_collection

import json
import numpy as np
from bert4keras.backend import keras, K, tf
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Layer, Dense, Permute
from keras.models import Model
from tqdm import tqdm

print(tf.__version__)
print(tf.test.is_gpu_available())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


1.15.2
True


In [ ]:
# 基本信息
maxlen = 512
epochs = 20
batch_size = 64
learing_rate = 2e-5


# 百度 阅读理解
train_data_file1 = '/content/drive/MyDrive/colab_data/data/dureader_robust-data/train.json'  
eva_data_file = '/content/drive/MyDrive/colab_data/data/dureader_robust-data/dev.json'
eva_script = '/content/drive/MyDrive/colab_data/packages/albert_QA_colab/evaluate/evaluate_dureader.py'

# CMRC2018
train_data_file2 = '/content/drive/MyDrive/colab_data/data/cmrc2018/cmrc2018_train_dev.json'
#eva_data_file = '/content/drive/MyDrive/colab_data/data/cmrc2018/cmrc2018_trial.json'
#eva_script = '/content/drive/MyDrive/colab_data/packages/albert_QA_colab/evaluate/evaluate_cmrc2018.py'

'''
bert4keras 支持的 BERT model_type
    'bert': BERT,
    'albert': ALBERT,
    'albert_unshared': ALBERT_Unshared,
    'roberta': BERT,
'''

# bert配置
#model_type = 'bert'
#config_path = '/content/drive/MyDrive/colab_data/models/bert_chinese_L-12_H-768_A-12/bert_config.json'
#checkpoint_path = '/content/drive/MyDrive/colab_data/models/bert_chinese_L-12_H-768_A-12/bert_model.ckpt'
#dict_path = '/content/drive/MyDrive/colab_data/models/bert_chinese_L-12_H-768_A-12/vocab.txt'

# albert配置
model_type = 'albert'
config_path = '/content/drive/MyDrive/colab_data/models/albert_base/albert_config.json'
checkpoint_path = '/content/drive/MyDrive/colab_data/models/albert_base/model.ckpt-best'
dict_path = '/content/drive/MyDrive/colab_data/models/albert_base/vocab_chinese.txt'

# 输出目录
output_path = '/content/drive/MyDrive/colab_data/outputs/keras_QA'
output_path = '%s/batch%d_max%d_lr%.0e'%(output_path, batch_size, maxlen, learing_rate)


# 兼容两个数据集的载入
def load_data(filename_list):
    D = []
    for filename in filename_list:
        for d in json.load(open(filename))['data']:
            for pp in d['paragraphs']:
                for qa in pp['qas']:
                    D.append([
                        qa['id'], pp['context'], qa['question'],
                        [a['text'] for a in qa.get('answers', [])]
                    ])
    return D


# 读取数据, 使用两个数据集一起训练
train_data = load_data([train_data_file1, train_data_file2])

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)


def search(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, item in self.sample(random):
            context, question, answers = item[1:]
            token_ids, segment_ids = tokenizer.encode(
                question, context, maxlen=maxlen
            )
            a = np.random.choice(answers)
            a_token_ids = tokenizer.encode(a)[0][1:-1]
            start_index = search(a_token_ids, token_ids)
            if start_index != -1:
                labels = [[start_index], [start_index + len(a_token_ids) - 1]]
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_labels.append(labels)
                if len(batch_token_ids) == self.batch_size or is_end:
                    batch_token_ids = sequence_padding(batch_token_ids)
                    batch_segment_ids = sequence_padding(batch_segment_ids)
                    batch_labels = sequence_padding(batch_labels)
                    yield [batch_token_ids, batch_segment_ids], batch_labels
                    batch_token_ids, batch_segment_ids, batch_labels = [], [], []


# 优化器，使用AMP
opt = Adam(learing_rate)
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

# 建立模型，载入权重
class MaskedSoftmax(Layer):
    """在序列长度那一维进行softmax，并mask掉padding部分
    """
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, 2)
            inputs = inputs - (1.0 - mask) * 1e12
        return K.softmax(inputs, 1)


model = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model=model_type
)


output = Dense(2)(model.output)
output = MaskedSoftmax()(output)
output = Permute((2, 1))(output)

model = Model(model.input, output)
model.summary()


def sparse_categorical_crossentropy(y_true, y_pred):
    # y_true需要重新明确一下shape和dtype
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    y_true = K.one_hot(y_true, K.shape(y_pred)[2])
    # 计算交叉熵
    return K.mean(K.categorical_crossentropy(y_true, y_pred))


def sparse_accuracy(y_true, y_pred):
    # y_true需要重新明确一下shape和dtype
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    # 计算准确率
    y_pred = K.cast(K.argmax(y_pred, axis=2), 'int32')
    return K.mean(K.cast(K.equal(y_true, y_pred), K.floatx()))


model.compile(
    loss=sparse_categorical_crossentropy,
    optimizer=opt,
    metrics=[sparse_accuracy]
)


def extract_answer(question, context, max_a_len=16):
    """抽取答案函数
    """
    max_q_len = 64
    q_token_ids = tokenizer.encode(question, maxlen=max_q_len)[0]
    c_token_ids = tokenizer.encode(
        context, maxlen=maxlen - len(q_token_ids) + 1
    )[0]
    token_ids = q_token_ids + c_token_ids[1:]
    segment_ids = [0] * len(q_token_ids) + [1] * (len(c_token_ids) - 1)
    c_tokens = tokenizer.tokenize(context)[1:-1]
    mapping = tokenizer.rematch(context, c_tokens)
    probas = model.predict([[token_ids], [segment_ids]])[0]
    probas = probas[:, len(q_token_ids):-1]
    start_end, score = None, -1
    for start, p_start in enumerate(probas[0]):
        for end, p_end in enumerate(probas[1]):
            if end >= start and end < start + max_a_len:
                if p_start * p_end > score:
                    start_end = (start, end)
                    score = p_start * p_end
    start, end = start_end
    return context[mapping[start][0]:mapping[end][-1] + 1]


def predict_to_file(infile, out_file):
    """预测结果到文件，方便提交
    """
    fw = open(out_file, 'w', encoding='utf-8')
    R = {}
    for d in tqdm(load_data([infile])):
        a = extract_answer(d[2], d[1])
        R[d[0]] = a
    R = json.dumps(R, ensure_ascii=False, indent=4)
    fw.write(R)
    fw.close()


def evaluate(filename):
    """评测函数（官方提供评测脚本evaluate.py）
    """
    predict_to_file(filename, 'evaluate.pred.json')
    metrics = json.loads(
        os.popen(
            'python3 %s %s %s'
            % (eva_script, filename, 'evaluate.pred.json')
        ).read().strip()
    )
    return metrics


class Evaluator(keras.callbacks.Callback):
    """评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        metrics = evaluate(eva_data_file)
        if float(metrics['F1']) >= self.best_val_f1:
            self.best_val_f1 = float(metrics['F1'])
            best_path = '%s_F1_%s'%(output_path,metrics['F1'])
            os.makedirs(best_path, exist_ok=True)
            model.save_weights('%s/best_model.weights' % best_path)
        metrics['BEST F1'] = self.best_val_f1
        print(metrics)

# main()
print("maxlen: ", maxlen)
print("epochs: ", epochs)
print("batch_size: ", batch_size)
print("learing_rate: ", learing_rate)
print("train data: ", len(train_data))

train_generator = data_generator(train_data, batch_size)
evaluator = Evaluator()

model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, None)]       0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    2704384     Input-Token[0][0]                
_________________________________________________________________________________________

100%|██████████| 1417/1417 [01:05<00:00, 21.55it/s]


{'F1': '79.521', 'EM': '66.690', 'TOTAL': 1417, 'SKIP': 0, 'BEST F1': 79.521}
436/436 [==============================] - 1986s 5s/step - loss: 1.9731 - sparse_accuracy: 0.5064
Epoch 2/20
435/436 [============================>.] - ETA: 4s - loss: 1.0199 - sparse_accuracy: 0.6880

100%|██████████| 1417/1417 [01:03<00:00, 22.42it/s]


{'F1': '82.000', 'EM': '69.725', 'TOTAL': 1417, 'SKIP': 0, 'BEST F1': 82.0}
436/436 [==============================] - 1962s 5s/step - loss: 1.0200 - sparse_accuracy: 0.6879
Epoch 3/20
435/436 [============================>.] - ETA: 4s - loss: 0.7861 - sparse_accuracy: 0.7498

100%|██████████| 1417/1417 [01:06<00:00, 21.37it/s]


{'F1': '81.366', 'EM': '68.948', 'TOTAL': 1417, 'SKIP': 0, 'BEST F1': 82.0}
436/436 [==============================] - 1958s 4s/step - loss: 0.7854 - sparse_accuracy: 0.7499
Epoch 4/20
300/436 [===================>..........] - ETA: 9:48 - loss: 0.6250 - sparse_accuracy: 0.7994